<h1> Satellite Precipitation Data Analysis</h1>

<p> This project's goal is to analyze PERSIANN (Precipitation Estimation from Remotely Sensed Information using Artificial Neural Networks) 
dataset provided by CHRS Data Portal. </p>

<h2>Data preparation</h2>

// TO DO